<a href="https://www.kaggle.com/code/rimzakhama/rsna-inference?scriptVersionId=143880904" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Install timm package from local directory
!pip install '/kaggle/input/timm-pack/timm_package/timm-0.6.12-py3-none-any.whl'

In [ ]:
# The medical images provided are in (two types of) Dicom format, and in order to read these 
# we will need to install and import the required dependent libraries:

'''
## If Online : install required packages for dcm processing 
!pip install -qU pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm dicomsdl

#import all the required dicom packages
import gdcm
import pydicom
import pylibjpeg
import dicomsdl
'''

!pip install pylibjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pylibjpeg-openjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pylibjpeg-libjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pydicom --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install python-gdcm --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install dicomsdl --no-index --find-links=file:///kaggle/input/read-dicom-set

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torch.nn as nn
from torchvision import transforms

import os

#import pydicom as dicom
import pydicom
import cv2
import PIL
from PIL import Image

import timm
import torch.optim as optim
from sklearn import model_selection
from sklearn.metrics import f1_score

from tqdm.autonotebook import tqdm
#from torchvision import models

from joblib import Parallel, delayed



import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(42)

In [ ]:
class Config :
    BATCH_SIZE = 1 #1 works, 16 threws exception
    #MODEL_PATH = '/kaggle/input/efficientb4-model/model.bin'
    # if resnet
    #MODEL_PATH = '/kaggle/input/resnet18/model.bin'
    
    # if efficientnet 
    MODEL_PATH = '/kaggle/input/tf_efficientnetv2_s/model.bin'

    NUM_CLASSES = 1

In [ ]:
class Dataset:
    def __init__(self, df, transform=None):
        self.df = df.copy()
        self.transform = transform
     
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        patient_id = self.df.loc[idx, 'patient_id']
        image_id = self.df.loc[idx, 'image_id']
        prediction_id = self.df.loc[idx, 'prediction_id']
        
        # Get and preprocess images
        # This path should be changed
        #png_path = '/kaggle/input/rsna-png-images-same-format-as-original/output/rsna_pngs/train_images'
        #png_path = '/kaggle/working/output/test_images' # test images path
        png_path = "./test_images"
        # Image path
        image_png_path =  os.path.join(png_path, patient_id.astype(str), image_id.astype(str)+'.png')

        # convert image to RGB
        image = Image.open(image_png_path).convert('RGB')
        
        # Check if image is RGB 
        #print('image shape = ', np.array(image).shape)
        
        image = image.resize((1024,912))
        
        # Apply transformers on images
        if self.transform:
            image = self.transform(image).to(torch.float32)
        
        # Convert to tensors
        #image = torch.tensor(image, dtype=torch.long)
        
        
        
        #{ 'image' : torch.tensor(image, dtype=torch.long),
         #        'prediction_id' : prediction_id
          #         }
        
        return image, prediction_id

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        self.network = timm.create_model('resnet18', pretrained=False, in_chans=3)
        n_features = self.network.fc.out_features
        
        # add additional layer that maps 2048 extracted features from resnet to 1 feature 
        #determining the class
        self.classifier_layer = nn.Sequential(
            nn.Linear(n_features , 256),
            nn.Dropout(0.), # 0.3 in the training part, There is no dropout in the inference part
            #Dropout with p=0 is equivalent to the identity operation
            nn.Linear(256 , 1)
        )
    
    def forward(self, xb):        
        xb = self.network(xb)
        xb = self.classifier_layer(xb)
        return torch.sigmoid(xb) #xb

In [ ]:
class BreastCancerModel(nn.Module):
    def __init__(self, Config):
        super().__init__()
        self.efficientnet = timm.create_model('efficientnet_b4', pretrained=False,
                                             in_chans=1)
        
        in_features = self.efficientnet.classifier.in_features
        self.efficientnet.classifier = nn.Linear(in_features, Config.NUM_CLASSES)   
        
    def forward(self, image):
        output = self.efficientnet(image)
        
        return output


In [ ]:
# Efficientnetv2_s
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (
            self.__class__.__name__
            + "("
            + "p="
            + "{:.4f}".format(self.p.data.tolist()[0])
            + ", "
            + "eps="
            + str(self.eps)
            + ")"
        )

class Efficientnetv2_s(nn.Module):
    def __init__(self, p=3, p_trainable=False, eps=1e-6):
        super(Efficientnetv2_s, self).__init__()
        
        # tf_efficientnetv2_s
        self.efficientnetv2_s = timm.create_model('tf_efficientnetv2_s', pretrained=False, in_chans=3)
        model = self.efficientnetv2_s
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        self.fc = nn.Linear(n_features, 1) # cancer
        self.pool = nn.Sequential(
            GeM(p=p, eps=eps, p_trainable=p_trainable),
            nn.Flatten())
    
    # if tf_efficientnetv2_s
    def forward(self, x):
        x = self.efficientnetv2_s(x) 
        #x = self.efficientnetv2_s.forward_features(x)
        #x = self.pool(x)
        logits = self.fc(x)
        return logits   

In [ ]:
class Models(nn.Module):
    def __init__(self):
        super(Models, self).__init__()
        #self.network = models.resnet18(pretrained=True)
        #n_features = self.network.fc.out_features
        
        # Resnet
        self.resnet = timm.create_model('resnet18', pretrained=False, in_chans=3)
        n_features_resnet = self.resnet.fc.out_features
        
        self.classifier_layer_resnet = nn.Sequential(
            nn.Linear(n_features_resnet , 256),
            nn.Dropout(0.8), # 0.3
            nn.Linear(256 , 1) #NUM_CLASSES = 1
        )
        
        # Efficientnet
        #self.efficientnet = timm.create_model('tf_efficientnetv2_s', pretrained=True, in_chans=3)
        self.efficientnet = timm.create_model('efficientnet_b4', pretrained=False, in_chans=3)
        in_features_efficientnet = self.efficientnet.classifier.in_features
        
        self.classifier_layer_efficientnet = nn.Sequential(
            nn.Linear(in_features_efficientnet , 256),
            nn.Dropout(0.5),
            nn.Linear(256 , 1)
        )
        
        # tf_efficientnetv2_s
        self.model = timm.create_model('tf_efficientnetv2_s', pretrained=False, in_chans=3)
        model = self.model
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        self.fc = nn.Linear(n_features, 1) # cancer
        #self.pool = nn.Sequential(
        #    GeM(p=p, eps=eps, p_trainable=p_trainable),
        #    nn.Flatten())
    
    # if tf_efficientnetv2_s
    def forward(self, x):
        x = self.model(x)
        #x = self.model.forward_features(x)
        #x = self.pool(x)
        logits = self.fc(x)
        return logits   
        


# Preprocess test images (convert to png format and cut off empty space)

test_dcm_images_path = '/kaggle/input/rsna-breast-cancer-detection/test_images'
patient = '10008'

patient_input_path = test_dcm_images_path + '/' + patient

# Convert test images to png
#output_path =  '/kaggle/working/output/rsna_pngs/test_images'
output_path =  '/kaggle/working/output/test_images'

# Create directory if it does not exists
if not os.path.isdir(output_path+'/'+patient):
    os.makedirs(output_path+'/'+patient)

for image in os.listdir(patient_input_path):
    image_input_path = patient_input_path + '/'+ image
    dicom_image = pydicom.dcmread(image_input_path)
    image_array = dicom_image.pixel_array
    scaled_img = (np.maximum(image_array,0) / image_array.max()) * 255.0
    if dicom_image.PhotometricInterpretation == "MONOCHROME1":
        scaled_img = 1 - scaled_img
    img = scaled_img.astype(np.uint8)
    resized_image = cv2.resize(img, (1024, 912))
    #cv2.imwrite(output_path + '/' + patient + '/' + image.replace('dcm', 'png'),resized_image)
    
    # 2. Crop
    X = resized_image
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    
    
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 0.05).astype(np.uint8)[:, :], 8, cv2.CV_32S)

    # stats.shape == (N, 5), where N is the number of regions, 5 dimensions correspond to:
    # left, top, width, height, area_size
    stats = output[2]
    
    # finding max area which always corresponds to the breast data. 
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    
    # cutting out the breast data
    X_fit = X[y1: y2, x1: x2]
    
    cv2.imwrite(output_path + '/' + patient + '/' + image.replace('dcm', 'png'),X_fit)
    

    # Plot png image
    #img = mpimg.imread(output_path + '/' + patient + '/' + image.replace('dcm', 'png'))
    #imgplot = plt.imshow(img)
    #plt.show()

    


In [ ]:
class CFG:
    seed = 42
    resize_dim = 1024
    aspect_ratio = True
    img_size = (1024, 912)
    #batch_size = 16
    
    # Model
    #model_name = "efficientnet_b4"
    num_classes = 1
    n_channels = 3
    
    test_img_path = "./test_images"

    
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import re
import pydicom
import glob
from PIL import Image

def fit_image(fname, size=1024):
    # 1. Read, resize
    patient = fname.split('/')[-2]
    image = fname.split('/')[-1][:-4]
    dicom = pydicom.dcmread(fname)
    img = dicom.pixel_array
    img = (img - img.min()) / (img.max() - img.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img
    #img = cv2.resize(img, (512, 512))
    img = cv2.resize(img, (1024, 912))

    
    # 2. Crop
    X = img
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    
    
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 0.05).astype(np.uint8)[:, :], 8, cv2.CV_32S)

    # stats.shape == (N, 5), where N is the number of regions, 5 dimensions correspond to:
    # left, top, width, height, area_size
    stats = output[2]
    
    # finding max area which always corresponds to the breast data. 
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    
    # cutting out the breast data
    X_fit = X[y1: y2, x1: x2]
    
    patient_id, im_id = os.path.basename(os.path.dirname(fname)), os.path.basename(fname)[:-4]
    os.makedirs(f'{CFG.test_img_path}/{patient_id}', exist_ok=True)
    cv2.imwrite(f'{CFG.test_img_path}/{patient_id}/{im_id}.png', (X_fit[:, :] * 255).astype(np.uint8))

    

def fit_all_images(all_images):
    with ThreadPoolExecutor(2) as p:
        for i in tqdm(p.map(fit_image, all_images), total=len(all_images)):
            pass

all_images = glob.glob('/kaggle/input/rsna-breast-cancer-detection/test_images/*/*') 
# all_images = glob.glob('/kaggle/input/rsna-breast-cancer-detection/train_images/10006/*')
fit_all_images(all_images)


In [ ]:
!find test_images | head

In [ ]:
dfx = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#model = BreastCancerModel(Config=Config)
model = Efficientnetv2_s() # from RSNA_my_version_with_train_test_split
#model = Models() # from Fork of work of 7th place_try some parts of code

model.to(device)

# Load the checkpoint
PATH = '/kaggle/input/tf-efficientnetv2-s/model.pt'
#PATH = '/kaggle/input/efficient-model/models/efficientv5_seed_10.pth'

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
#model.load_state_dict(checkpoint['model'])

#threshold = checkpoint['threshold']
threshold = 0.5

#model.load_state_dict(torch.load(Config.MODEL_PATH))

model.eval()

transform= transforms.Compose([
    # input for augmentator is always PIL image
    # transforms.ToPILImage(),
    #transforms.RandomHorizontalFlip(0.5),
    #transforms.RandomVerticalFlip(0.5),
    #transforms.RandomPerspective(),
    #transforms.RandomRotation((0, 90)),
    #transforms.RandomAutocontrast(),
    #transforms.RandomAffine(degrees=(0, 180), scale=(0.8, 1.2)),
    #transforms.ElasticTransform(),
    transforms.ToTensor(), # return it as a tensor and transforms it to [0, 1]
    transforms.Normalize(mean = [0.1338, 0.1338, 0.1338],
                         std = [0.2068, 0.2068, 0.2068])    
])

dataset = Dataset(dfx, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size= Config.BATCH_SIZE,
                                             num_workers=0)

with torch.no_grad():
    predictions = []
    
    
    for batch_idx, (images, prediction_id) in enumerate(data_loader):
        images = images.to(device, dtype=torch.float) 
        logits = model(images).squeeze()
        prob_preds = logits.sigmoid() 
        predictions.append(prob_preds.cpu())
        
        #print('predictions before stack =', predictions)
        
    predictions = torch.stack(predictions, dim=0).numpy()   
    #print('predictions after stack =', predictions)

predictions = predictions.reshape(-1) # convert nd array to 1d array
#print('prediction type = ', type(predictions))
#print('predictions shape = ', predictions.shape)
#print('length of predictions = ', len(predictions))



dfx['cancer'] = predictions

sample = dfx.groupby('prediction_id')[['cancer']].max()#.reset_index()

#sample['cancer'] = (sample.cancer > threshold.detach().numpy()).astype(int)
sample['cancer'] = (sample.cancer > 0.5).astype(int)

# Save sample DataFrame as csv
sample.to_csv('submission.csv', index=True)
!head submission.csv
#sample.head()